In [24]:
import pandas as pd
import glob,os
import re

OMOP_FILES_PATH = 'codelists/OMOP/'
OUTPUT_PATH = 'codelists/output/omop/uk'

omop_concepts  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT.csv', sep='\t')
omop_relationships  = pd.read_csv(OMOP_FILES_PATH+'RELATIONSHIP.csv', sep='\t')
omop_concept_relationships  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT_RELATIONSHIP.csv', sep='\t')


omop_concepts

/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_13472/1901784593.py:8: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  omop_concepts  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT.csv', sep='\t')


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,1146945,concept.concept_id,Metadata,CDM,Field,S,CDM1,20141111,20991231,NaN
1,1146954,concept.invalid_reason,Metadata,CDM,Field,S,CDM10,20141111,20991231,NaN
2,1147044,observation_period.observation_period_id,Metadata,CDM,Field,S,CDM100,20141111,20991231,NaN
3,756315,metadata.metadata_type_concept_id,Metadata,CDM,Field,S,CDM1000,20210925,20991231,NaN
4,756316,metadata.name,Metadata,CDM,Field,S,CDM1001,20210925,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
1473553,33091,IQVIA CO-CONNECT,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473554,33092,IQVIA CO-CONNECT MIABIS,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473555,33093,IQVIA CO-CONNECT TWINS,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473556,33095,NHS Ethnic Category,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN


In [25]:
s='colorectal'
files = sorted( list( glob.glob("codelists/output/uk/query_"+s+"*_snomed.csv") ) )
files

['codelists/output/uk/query_colorectal_0_snomed.csv',
 'codelists/output/uk/query_colorectal_1_snomed.csv',
 'codelists/output/uk/query_colorectal_2_snomed.csv',
 'codelists/output/uk/query_colorectal_3_snomed.csv']

In [26]:
files = ['codelists/output/uk/query_colorectal_0_snomed.csv']

In [27]:
omop_concept_relationships['relationship_id']

0           Has status
1           Has Module
2           Has status
3           Has Module
4           Has status
               ...    
18893613    Has Module
18893614    Has status
18893615    Has Module
18893616    Has status
18893617    Has Module
Name: relationship_id, Length: 18893618, dtype: object

In [33]:
for f in files:
    name = re.split('/|\.', f)[-2]
    input_snomed_list = pd.read_csv(f)

    input_snomed_list['concept_id'] = input_snomed_list['concept_id'].astype(str)
    omop_concepts['concept_code'] = omop_concepts['concept_code'].astype(str)
    omop_concepts['concept_id'] = omop_concepts['concept_id'].astype(str)
    omop_concept_relationships['concept_id_2'] = omop_concept_relationships['concept_id_2'].astype(str)
    omop_concept_relationships['concept_id_1'] = omop_concept_relationships['concept_id_1'].astype(str)


    omop_concepts_filtered = omop_concepts[ omop_concepts['vocabulary_id'] =='SNOMED' ]
    omop_concepts_filtered = omop_concepts_filtered[ omop_concepts_filtered['concept_code'].isin(input_snomed_list['concept_id']) ]

    omop_concepts_filtered = omop_concepts_filtered[['concept_id','concept_name','vocabulary_id','concept_code']]


    concept_maps = omop_concepts_filtered.merge( omop_concept_relationships, how='left', left_on='concept_id', right_on='concept_id_1' )
    concept_maps_named = concept_maps.merge(omop_concepts, how='left', left_on='concept_id_2', right_on='concept_id', suffixes=('_snomed','_opcs') )
    concept_maps_named = concept_maps_named[ concept_maps_named['vocabulary_id_opcs']=='OPCS4' ]

    display(concept_maps_named)

    snomed_to_opcs = concept_maps_named[['concept_id_snomed','concept_name_snomed','concept_code_snomed','relationship_id','concept_id_opcs','concept_name_opcs','concept_code_opcs']]



,concept_id,concept_name,vocabulary_id,concept_code
428929,4010018,Proctopexy,SNOMED,10259007
433234,36712674,Hot biopsy,SNOMED,10361000132103
433667,36712675,Colonoscopic snare polypectomy of colon,SNOMED,10371000132109
438933,4020571,Radionuclide therapy for glandular suppression,SNOMED,10492003
439784,507670,Low dose CT colonography,SNOMED,1051291000000100
...,...,...,...,...
1441311,4241048,Colotomy,SNOMED,9215005
1444134,4245100,Proctoplasty for stenosis,SNOMED,9280007
1454239,4316867,"Anal fistulectomy, second stage",SNOMED,9521005
1459852,4321434,"Esophagocolic anastomosis, antesternal or ante...",SNOMED,9666005


,concept_id,concept_name,vocabulary_id,concept_code,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,4010018,Proctopexy,SNOMED,10259007,4010018,3532347,Concept was_a from,20111003,20991231,NaN
1,4010018,Proctopexy,SNOMED,10259007,4010018,3532347,Mapped from,20200731,20991231,NaN
2,4010018,Proctopexy,SNOMED,10259007,4010018,40337896,Concept poss_eq from,20140401,20991231,NaN
3,4010018,Proctopexy,SNOMED,10259007,4010018,40642537,Has status,20220128,20991231,NaN
4,4010018,Proctopexy,SNOMED,10259007,4010018,40642539,Has Module,20220128,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
19336,4319900,Repair of colon,SNOMED,9735005,4319900,4312203,Subsumes,20110731,20991231,NaN
19337,4319900,Repair of colon,SNOMED,9735005,4319900,4319900,Mapped from,19700101,20991231,NaN
19338,4319900,Repair of colon,SNOMED,9735005,4319900,4319900,Maps to,19700101,20991231,NaN
19339,4319900,Repair of colon,SNOMED,9735005,4319900,40485319,Subsumes,20110731,20991231,NaN


,concept_id_snomed,concept_name_snomed,vocabulary_id_snomed,concept_code_snomed,concept_id_1,concept_id_2,relationship_id,valid_start_date_snomed,valid_end_date_snomed,invalid_reason_snomed,concept_id_opcs,concept_name_opcs,domain_id,vocabulary_id_opcs,concept_class_id,standard_concept,concept_code_opcs,valid_start_date_opcs,valid_end_date_opcs,invalid_reason_opcs
51,4010018,Proctopexy,SNOMED,10259007,4010018,44510846,Subsumes,20180829,20991231,NaN,44510846,Other specified fixation of rectum for prolapse,Procedure,OPCS4,Procedure,S,H35.8,19700101,20991231,NaN
52,4010018,Proctopexy,SNOMED,10259007,4010018,44510847,Mapped from,20180829,20991231,NaN,44510847,Unspecified fixation of rectum for prolapse,Procedure,OPCS4,Procedure,NaN,H35.9,19700101,20991231,NaN
53,4010018,Proctopexy,SNOMED,10259007,4010018,46232531,Mapped from,20180829,20991231,NaN,46232531,Fixation of rectum for prolapse,Procedure,OPCS4,Procedure,NaN,H35,20150401,20991231,NaN
54,4010018,Proctopexy,SNOMED,10259007,4010018,46232532,Mapped from,20180829,20991231,NaN,46232532,Other abdominal operations for prolapse of rectum,Procedure,OPCS4,Procedure,NaN,H36,20150401,20991231,NaN
1269,4031507,Ileotransversostomy,SNOMED,14403007,4031507,44510574,Mapped from,20180829,20991231,NaN,44510574,Anastomosis of ileum to transverse colon,Procedure,OPCS4,Procedure,NaN,G72.2,19700101,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19162,44810474,Repositioning of colostomy,SNOMED,898171000000107,44810474,44510823,Subsumes,20180829,20991231,NaN,44510823,Resiting of colostomy,Procedure,OPCS4,Procedure,S,H32.1,19700101,20991231,NaN
19257,4241048,Colotomy,SNOMED,9215005,4241048,44510730,Mapped from,20180829,20991231,NaN,44510730,Colotomy,Procedure,OPCS4,Procedure,NaN,H16.3,19700101,20991231,NaN
19258,4241048,Colotomy,SNOMED,9215005,4241048,44510731,Subsumes,20180829,20991231,NaN,44510731,Other specified incision of colon,Procedure,OPCS4,Procedure,S,H16.8,19700101,20991231,NaN
19259,4241048,Colotomy,SNOMED,9215005,4241048,44510732,Mapped from,20180829,20991231,NaN,44510732,Unspecified incision of colon,Procedure,OPCS4,Procedure,NaN,H16.9,19700101,20991231,NaN


In [35]:
snomed_to_opcs.drop_duplicates()

,concept_id_snomed,concept_name_snomed,concept_code_snomed,relationship_id,concept_id_opcs,concept_name_opcs,concept_code_opcs
51,4010018,Proctopexy,10259007,Subsumes,44510846,Other specified fixation of rectum for prolapse,H35.8
52,4010018,Proctopexy,10259007,Mapped from,44510847,Unspecified fixation of rectum for prolapse,H35.9
53,4010018,Proctopexy,10259007,Mapped from,46232531,Fixation of rectum for prolapse,H35
54,4010018,Proctopexy,10259007,Mapped from,46232532,Other abdominal operations for prolapse of rectum,H36
1269,4031507,Ileotransversostomy,14403007,Mapped from,44510574,Anastomosis of ileum to transverse colon,G72.2
...,...,...,...,...,...,...,...
19162,44810474,Repositioning of colostomy,898171000000107,Subsumes,44510823,Resiting of colostomy,H32.1
19257,4241048,Colotomy,9215005,Mapped from,44510730,Colotomy,H16.3
19258,4241048,Colotomy,9215005,Subsumes,44510731,Other specified incision of colon,H16.8
19259,4241048,Colotomy,9215005,Mapped from,44510732,Unspecified incision of colon,H16.9


In [43]:
method_relationships = omop_concept_relationships[ omop_concept_relationships['relationship_id']=='Has method' ]
method_relationships

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
14106,4237501,4045073,Has method,20220128,20991231,NaN
20970,619843,40293773,Has method,20220128,20991231,NaN
97320,36674184,4044176,Has method,20190131,20991231,NaN
97337,36674186,4081836,Has method,20190131,20991231,NaN
97339,36674186,4106470,Has method,20190131,20991231,NaN
...,...,...,...,...,...,...
18892362,619793,4044525,Has method,20220128,20991231,NaN
18892369,619794,4044525,Has method,20220128,20991231,NaN
18892385,619795,40293775,Has method,20220128,20991231,NaN
18892400,618616,40293773,Has method,20220128,20991231,NaN


In [47]:
snomed_opcs_methods = snomed_to_opcs.merge( method_relationships, how='left', left_on='concept_id_snomed', right_on='concept_id_1' )

snomed_opcs_methods_named = snomed_opcs_methods.merge(omop_concepts, how='left', left_on='concept_id_2', right_on='concept_id')
snomed_opcs_methods_named

,concept_id_snomed,concept_name_snomed,concept_code_snomed,relationship_id_x,concept_id_opcs,concept_name_opcs,concept_code_opcs,concept_id_1,concept_id_2,relationship_id_y,...,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date_y,valid_end_date_y,invalid_reason_y
0,4010018,Proctopexy,10259007,Subsumes,44510846,Other specified fixation of rectum for prolapse,H35.8,4010018,4045069,Has method,...,4045069,Fixation - action,Observation,SNOMED,Qualifier Value,S,129371009,20020131.0,20991231.0,NaN
1,4010018,Proctopexy,10259007,Mapped from,44510847,Unspecified fixation of rectum for prolapse,H35.9,4010018,4045069,Has method,...,4045069,Fixation - action,Observation,SNOMED,Qualifier Value,S,129371009,20020131.0,20991231.0,NaN
2,4010018,Proctopexy,10259007,Mapped from,46232531,Fixation of rectum for prolapse,H35,4010018,4045069,Has method,...,4045069,Fixation - action,Observation,SNOMED,Qualifier Value,S,129371009,20020131.0,20991231.0,NaN
3,4010018,Proctopexy,10259007,Mapped from,46232532,Other abdominal operations for prolapse of rectum,H36,4010018,4045069,Has method,...,4045069,Fixation - action,Observation,SNOMED,Qualifier Value,S,129371009,20020131.0,20991231.0,NaN
4,4031507,Ileotransversostomy,14403007,Mapped from,44510574,Anastomosis of ileum to transverse colon,G72.2,4031507,4044182,Has method,...,4044182,Incision - action,Observation,SNOMED,Qualifier Value,S,129287005,20020131.0,20991231.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,44810474,Repositioning of colostomy,898171000000107,Subsumes,44510823,Resiting of colostomy,H32.1,44810474,4232676,Has method,...,4232676,Repositioning - action,Observation,SNOMED,Qualifier Value,S,360146007,20020131.0,20991231.0,NaN
390,4241048,Colotomy,9215005,Mapped from,44510730,Colotomy,H16.3,4241048,4044182,Has method,...,4044182,Incision - action,Observation,SNOMED,Qualifier Value,S,129287005,20020131.0,20991231.0,NaN
391,4241048,Colotomy,9215005,Subsumes,44510731,Other specified incision of colon,H16.8,4241048,4044182,Has method,...,4044182,Incision - action,Observation,SNOMED,Qualifier Value,S,129287005,20020131.0,20991231.0,NaN
392,4241048,Colotomy,9215005,Mapped from,44510732,Unspecified incision of colon,H16.9,4241048,4044182,Has method,...,4044182,Incision - action,Observation,SNOMED,Qualifier Value,S,129287005,20020131.0,20991231.0,NaN


In [84]:
aggregation_map = snomed_opcs_methods_named[['concept_name_snomed','concept_code_opcs','concept_name_opcs','concept_name']].drop_duplicates()
aggregation_map

,concept_name_snomed,concept_code_opcs,concept_name_opcs,concept_name
0,Proctopexy,H35.8,Other specified fixation of rectum for prolapse,Fixation - action
1,Proctopexy,H35.9,Unspecified fixation of rectum for prolapse,Fixation - action
2,Proctopexy,H35,Fixation of rectum for prolapse,Fixation - action
3,Proctopexy,H36,Other abdominal operations for prolapse of rectum,Fixation - action
4,Ileotransversostomy,G72.2,Anastomosis of ileum to transverse colon,Incision - action
...,...,...,...,...
389,Repositioning of colostomy,H32.1,Resiting of colostomy,Repositioning - action
390,Colotomy,H16.3,Colotomy,Incision - action
391,Colotomy,H16.8,Other specified incision of colon,Incision - action
392,Colotomy,H16.9,Unspecified incision of colon,Incision - action


In [85]:
counts = aggregation_map.groupby('concept_code_opcs').size()
counts.value_counts()

1    270
2     59
3      2
Name: count, dtype: int64

In [86]:
duplicate_maps = aggregation_map[ aggregation_map['concept_code_opcs'].isin(counts[ counts > 1 ].index) ]
duplicate_maps

,concept_name_snomed,concept_code_opcs,concept_name_opcs,concept_name
7,"Hartmann operation, rectal resection",H33.5,Rectosigmoidectomy and closure of rectal stump...,Excision - action
8,"Hartmann operation, rectal resection",H33.5,Rectosigmoidectomy and closure of rectal stump...,Closure - action
9,"Hartmann operation, rectal resection",H33.5,Rectosigmoidectomy and closure of rectal stump...,Construction - action
18,Total esophagectomy and interposition of micro...,G02.4,Total oesophagectomy and interposition of micr...,Excision - action
19,Total esophagectomy and interposition of micro...,G02.4,Total oesophagectomy and interposition of micr...,Interposition anastomosis - action
...,...,...,...,...
384,Colectomy and end to side anastomosis,H11.6,Colectomy and end to side anastomosis NEC,End-to-side anastomosis - action
386,Anterior resection of rectum with colostomy,H33.6,Anterior resection of rectum and exteriorisati...,Excision - action
387,Anterior resection of rectum with colostomy,H33.6,Anterior resection of rectum and exteriorisati...,Construction - action
388,Repositioning of colostomy,H32.1,Resiting of colostomy,Surgical action


In [53]:
aggregation_map['concept_name'].value_counts()

concept_name
Excision - action                       115
Anastomosis - action                     23
Exteriorization - action                 18
Repair - action                          18
Extirpation - action                     18
Destruction - action                     17
Administration - action                  13
Fixation - action                        10
Surgical action                          10
Incision - action                        10
Bypass - action                          10
Closure - action                          8
Inspection - action                       8
Manipulation - action                     7
End-to-side anastomosis - action          7
Dilation - action                         7
Radiographic imaging - action             7
Radioactive destruction                   5
End-to-end anastomosis - action           5
Construction - action                     5
Open reduction - action                   4
Manual reduction - action                 4
Brachytherapy - act

In [87]:
len(aggregation_map['concept_name'].value_counts())

48